In [26]:

from dataset_slide_turn_change_bl import *
import torch
import torch.nn as nn

import numpy as np

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
torch.manual_seed(0)

person_order = {'F1_Interaction_1': {'P2': 1, 'P1': 1, 'P3': 2},
 'F1_Interaction_2': {'P2': 1, 'P1': 1, 'P3': 2},
 'F2_Interaction_1': {'P4': 1, 'P5': 3},
 'F2_Interaction_2': {'P4': 1},
 'F3_Interaction_1': {'P8': 3, 'P6': 1, 'P7': 1},
 'F3_Interaction_2': {'P6': 1, 'P7': 1},
 'F4_Interaction_1': {'P14': 2,
  'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F4_Interaction_2': {'P12': 1,
  'P11': 1,
  'P10': 1,
  'P9': 1,
  'P13': 3},
 'F5_Interaction_1': {'P16': 2, 'P15': 1},
 'F5_Interaction_2': {'P16': 2, 'P15': 1},
 'F6_Interaction_1': {'P19': 3, 'P18': 1, 'P17': 1},
 'F6_Interaction_2': {'P19': 3, 'P18': 1, 'P17': 1},
 'F7_Interaction_1': {'P22': 3,
  'P20': 1,
  'P21': 1,
  'P23': 2},
 'F8_Interaction_1': {'P24': 1, 'P25': 3},
 'F8_Interaction_2': {'P24': 1, 'P25': 3},
 'F8_Interaction_3': {'P24': 1, 'P25': 3},
 'F10_Interaction_1': {'P27': 1, 'P28': 1},
 'F11_Interaction_1': {'P29': 1, 'P30': 2},
 'F11_Interaction_2': {'P29': 1, 'P30': 2},
 'F13_Interaction_1': {'P32': 1, 'P33': 2},
 'F17_Interaction_1': {'P37': 1, 'P38': 2},
 'F17_Interaction_2': {'P37': 1, 'P38': 2}}


group_nums = {1: ['F2_Interaction_2'],
 2: ['F2_Interaction_1',
  'F3_Interaction_2',
  'F5_Interaction_1',
  'F5_Interaction_2',
  'F8_Interaction_1',
  'F8_Interaction_2',
  'F8_Interaction_3',
  'F10_Interaction_1',
  'F11_Interaction_1',
  'F11_Interaction_2',
  'F13_Interaction_1',
  'F17_Interaction_1',
  'F17_Interaction_2'],
 3: ['F1_Interaction_1',
  'F1_Interaction_2',
  'F3_Interaction_1',
  'F6_Interaction_1',
  'F6_Interaction_2'],
 4: ['F7_Interaction_1'],
 5: ['F4_Interaction_2'],
 6: ['F4_Interaction_1']}

group_all_dataset = []
group_ids = group_nums[3]
for group_id in group_ids:
    group_specific_dataset = SpeedDatingDS(group_id = group_id, social_rel = person_order[group_id])
    group_all_dataset.append(group_specific_dataset)

SD = torch.utils.data.ConcatDataset(group_all_dataset)

########################################################################
#Dataloader
########################################################################
train_len = len(SD) - len(SD)//5
test_len = len(SD)//5

train, test = torch.utils.data.random_split(SD, (train_len, test_len), generator=torch.Generator().manual_seed(0))

batch_size = 32
trainloader = DataLoader(train, batch_size = train_len, shuffle = False, num_workers = 8)
testloader = DataLoader(test, batch_size = test_len, shuffle = False, num_workers = 8)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for idx, batch in enumerate(trainloader):

    x_train, vb_output = batch['context'], batch['vb_output']

    labels = vb_output.sum(2).to(device).flatten(start_dim =1)
    index_labels = torch.zeros(x_train.shape[0]).long().to(device)
    index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
    y_train = index_labels

for idx, batch in enumerate(testloader):
    x_test, vb_output = batch['context'], batch['vb_output']

    labels = vb_output.sum(2).to(device).flatten(start_dim =1)
    index_labels = torch.zeros(x_test.shape[0]).long().to(device)
    index_labels[labels.nonzero()[:,0]] = labels.nonzero()[:,1] + 1 
    y_test = index_labels




In [27]:
#gaze, prosody, head


desired_ind = [0,6] + [*range(29,38)]
x_abl = x_test[...,desired_ind].cpu().numpy()
y_abl = y_test.cpu().numpy()

In [28]:


y_pred = np.zeros(x_abl.shape[0])
for i, x_label in enumerate(x_test.sum(2).cpu().numpy()):
    print(i)
    
    if len(x_label.nonzero()[0]) > 0:
        
#         if len(x_label.nonzero()[0]) > 1:
#             pdb.set_trace()
#             print('here')
#             y_pred[i] = np.random.choice(x_label.nonzero()[0]) + 1 
#         else:
        
#         if y_test[i] == 2:
#             pdb.set_trace()
            
        y_pred[i] = int(x_label.nonzero()[0]) + 1
        
            

            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [30]:

print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_test.detach().cpu().numpy(), average='macro')))
print("weighted_f1: {}".format(sklearn.metrics.f1_score(y_pred, y_test.detach().cpu().numpy(), average='weighted')))
print("acc: {}".format((y_pred == y_test.detach().cpu().numpy()).mean()))
print(confusion_matrix(y_test.detach().cpu().numpy(), y_pred))

f1: 0.5686274509803921
weighted_f1: 0.6255835667600373
acc: 0.6
[[ 4  5  4]
 [ 1 12  1]
 [ 0  3  5]]


AttributeError: 'numpy.ndarray' object has no attribute 'mode'

In [33]:
values, counts = np.unique(y_test.detach().cpu().numpy(), return_counts=True)

ind = np.argmax(counts)
print(values[ind])  # prints the most frequent element

2


In [37]:
y_pred = np.zeros_like(y_test.detach().cpu().numpy()) + 2

In [38]:

print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_test.detach().cpu().numpy(), average='macro')))
print("weighted_f1: {}".format(sklearn.metrics.f1_score(y_pred, y_test.detach().cpu().numpy(), average='weighted')))
print("acc: {}".format((y_pred == y_test.detach().cpu().numpy()).mean()))
print(confusion_matrix(y_test.detach().cpu().numpy(), y_pred))

f1: 0.1904761904761905
weighted_f1: 0.5714285714285715
acc: 0.4
[[ 0 13  0]
 [ 0 14  0]
 [ 0  8  0]]


In [24]:


y_pred = np.zeros(x_test.shape[0])
for i, x_label in enumerate(x_test.sum(2)):
    
    if len(x_label.nonzero()[0]) > 0:
        
        if len(x_label.nonzero()[0]) > 1:
            pdb.set_trace()
            print('here')
            y_pred[i] = np.random.choice(x_label.nonzero()[0]) + 1 
        else:
            x_label.nonzero()[0]
            y_pred[i] = int(x_label.nonzero()[0]) + 1

In [25]:
y_pred

array([2., 3., 2., 2., 2., 1., 3., 1., 3., 2., 2., 3., 2., 3., 3., 3., 1.,
       2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 1., 2., 3., 2., 3., 1.,
       2.])

In [14]:
import sklearn

print("f1: {}".format(sklearn.metrics.f1_score(y_pred, y_test.detach().cpu().numpy(), average='macro')))
print("acc: {}".format((y_pred == y_test.detach().cpu().numpy()).mean()))
print(confusion_matrix(y_test.detach().cpu().numpy(), y_pred))

f1: 0.5686274509803921
acc: 0.6
[[ 4  5  4]
 [ 1 12  1]
 [ 0  3  5]]


In [ ]:
y_test